In [21]:
import numpy as np
import scipy as sp
import functools

In [39]:
nodes = []  
nodesnum = 0
nodes_index = {}  
damp = 0.8  
trans_matrix = np.matrix(0)  
sim_matrix = np.matrix(0)  
 
 
def initParam(graphFile):
   
    global nodes
    global nodes_index
    global trans_matrix
    global sim_matrix
    global damp
    global nodesnum
 
    link_in = {}
    for line in open(graphFile, "r", 1024):
        arr = line.strip("\n").split()
        node = arr[0]
        nodeid = -1
        if node in nodes_index:
            nodeid = nodes_index[node]
        else:
            nodeid = len(nodes)
            nodes_index[node] = nodeid
            nodes.append(node)
        for ele in arr[1:]:
            outneighbor = ele
            outneighborid = -1
            if outneighbor in nodes_index:
                outneighborid = nodes_index[outneighbor]
            else:
                outneighborid = len(nodes)
                nodes_index[outneighbor] = outneighborid
                nodes.append(outneighbor)
            inneighbors = []
            if outneighborid in link_in:
                inneighbors = link_in[outneighborid]
            inneighbors.append(nodeid)
            link_in[outneighborid] = inneighbors
 
    nodesnum = len(nodes)
    trans_matrix = np.zeros((nodesnum, nodesnum))
    for node, inneighbors in link_in.items():
        num = len(inneighbors)
        prob = 1.0 / num
        for neighbor in inneighbors:
            trans_matrix[neighbor, node] = prob
 
    sim_matrix = np.identity(nodesnum) * (1 - damp)
 
 
def iterate():

    global trans_matrix
    global sim_matrix
    global damp
    global nodesnum
 
    sim_matrix = damp * np.dot(np.dot(trans_matrix.transpose(),
                                      sim_matrix), trans_matrix) + (1 - damp) * np.identity(nodesnum)
 
 
def printResult(sim_node_file):
    
    global sim_matrix
    global link_out
    global link_in
    global nodes
    global nodesnum
    
    f_out_user = open(sim_node_file, "w")
    for i in range(nodesnum):
#         f_out_user.write(nodes[i] + "\t")
        neighbour = []
        for j in range(nodesnum):
            if i != j:
                sim = sim_matrix[i, j]
                if sim == None:
                    sim = 0
                if sim > 0:
                    neighbour.append((i, j, sim))
        for (v , u, sim) in neighbour:
            f_out_user.write(nodes[v] + ","+ nodes[u] + "," + str(sim) + "\t")
            f_out_user.write("\n")
    f_out_user.close()
 
 
def simrank(graphFile, maxIteration):
    global nodes_index
    global trans_matrix
    global sim_matrix
 
    initParam(graphFile)
    print( "nodes:")
    print(nodes_index)
    print("trans ratio:")
    print(trans_matrix)
    for i in range(maxIteration):
        print("iteration %d:" % (i + 1))
        iterate()
        print(sim_matrix)


In [40]:
if __name__ == '__main__':
    graphFile = "/Users/ninashao/Downloads/data_sample/MS_sample/relationship_total.txt"
    sim_node_file = "/Users/ninashao/Downloads/data_sample/MS_sample/nodesim_naive.txt"
    maxIteration = 10
    simrank(graphFile, maxIteration)
    printResult(sim_node_file)

nodes:
{'39803': 4032, '32767': 3172, '14654': 838, '25408': 2184, '27751': 2517, '16646': 1103, '41704': 4286, '24647': 2086, '35157': 3466, '39794': 4030, '33408': 3255, '24943': 2127, '26374': 2333, '40122': 4085, '15678': 983, '13888': 735, '28795': 2668, '17601': 1223, '11640': 420, '15615': 972, '24413': 2059, '33124': 3224, '22471': 1813, '33696': 3290, '11793': 447, '15863': 1005, '17435': 1199, '26196': 2301, '30553': 2907, '31051': 2956, '24991': 2136, '1274': 2629, '1222': 592, '27596': 2498, '1190': 391, '25651': 2223, '38820': 3912, '40603': 4145, '30242': 2868, '27052': 2426, '17625': 1227, '17585': 1219, '21723': 1714, '22781': 1855, '29452': 2748, '33716': 3295, '17975': 1267, '21232': 1650, '17895': 1255, '22816': 1860, '32475': 3135, '1165': 446, '16038': 1025, '11696': 431, '34457': 3368, '1003': 16, '38256': 3835, '28467': 2620, '18346': 1296, '40054': 4076, '23608': 1965, '24652': 2087, '15599': 967, '17970': 1265, '22022': 1750, '37240': 3717, '23498': 1951, '4114

[[  2.14545455e-01   0.00000000e+00   0.00000000e+00 ...,   3.63636364e-03
    9.69696970e-03   2.42424242e-03]
 [  0.00000000e+00   2.00567376e-01   2.57248513e-05 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   2.57248513e-05   2.00403023e-01 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 ..., 
 [  3.63636364e-03   0.00000000e+00   0.00000000e+00 ...,   2.20000000e-01
    3.33333333e-03   0.00000000e+00]
 [  9.69696970e-03   0.00000000e+00   0.00000000e+00 ...,   3.33333333e-03
    2.26666667e-01   4.44444444e-03]
 [  2.42424242e-03   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    4.44444444e-03   2.26666667e-01]]
iteration 2:
[[  2.14704797e-01   0.00000000e+00   0.00000000e+00 ...,   3.66648189e-03
    9.73384140e-03   2.44718568e-03]
 [  0.00000000e+00   2.04711432e-01   4.08524232e-03 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   4.08524232e-03   2.04444653e-01 ...,   0.00000000e+00
    0.00